In [1]:
from models.DAX.DAX_models import DAX_quantile_regression, DAX_baseline
from models.energy.energy_models import energy_quantile_regression, energy_LSTM, energy_XGBoost
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax

In [2]:
from functions import naive_ensemble

In [3]:
daxdata = get_DAX.get()

In [4]:
base = DAX_baseline.DAX_baseline(daxdata)

In [5]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,DAX,1 day,-3.321265,-0.543724,0.065019,0.710654,2.719772
1,2024-01-17,DAX,2 day,-4.087895,-0.799977,0.080265,0.976061,3.570492
2,2024-01-17,DAX,5 day,-5.259161,-0.896702,0.161798,1.224285,4.628697
3,2024-01-17,DAX,6 day,-6.282629,-1.007463,0.187498,1.435857,5.367396
4,2024-01-17,DAX,7 day,-6.735742,-1.054012,0.326190,1.557158,6.259934


In [6]:
df_dax=base

In [7]:
energydata= get_energy.get()

  0%|          | 0/265 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 265/265 [00:29<00:00,  9.08it/s]


In [8]:
df_energy= energy_XGBoost.energy_forecast(energydata,date_str='2024-01-17')

In [9]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,energy,36 hour,64.057678,70.027924,70.872932,75.010704,77.218788
1,2024-01-17,energy,40 hour,61.880836,66.906807,68.240898,72.040749,73.287773
2,2024-01-17,energy,44 hour,60.779305,62.762531,64.174416,68.024254,68.785774
3,2024-01-17,energy,60 hour,57.528931,60.668816,61.805847,64.520790,64.131546
4,2024-01-17,energy,64 hour,54.591389,57.760727,58.559002,60.839371,62.414742
5,2024-01-17,energy,68 hour,53.390141,56.549107,58.772770,59.213043,59.346821


In [10]:
df = merge_submissions.merge(df_energy,df_dax)

         forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0           2024-01-17         DAX    1 day  -3.321265  -0.543724   0.065019   
1           2024-01-17         DAX    2 day  -4.087895  -0.799977   0.080265   
2           2024-01-17         DAX    5 day  -5.259161  -0.896702   0.161798   
3           2024-01-17         DAX    6 day  -6.282629  -1.007463   0.187498   
4           2024-01-17         DAX    7 day  -6.735742  -1.054012    0.32619   
0  2024-01-17 00:00:00      energy  36 hour  64.057678  70.027924  70.872932   
1  2024-01-17 00:00:00      energy  40 hour  61.880836  66.906807  68.240898   
2  2024-01-17 00:00:00      energy  44 hour  60.779305  62.762531  64.174416   
3  2024-01-17 00:00:00      energy  60 hour  57.528931  60.668816  61.805847   
4  2024-01-17 00:00:00      energy  64 hour  54.591389  57.760727  58.559002   
5  2024-01-17 00:00:00      energy  68 hour  53.390141  56.549107   58.77277   
0           2024-01-17  infections   0 w

In [12]:
import pandas as pd

In [13]:
# ensure forecast_date is in correct format
df['forecast_date'] = pd.to_datetime(df['forecast_date'], format='%Y-%m-%d')

In [14]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,DAX,1 day,-3.321265,-0.543724,0.065019,0.710654,2.719772
1,2024-01-17,DAX,2 day,-4.087895,-0.799977,0.080265,0.976061,3.570492
2,2024-01-17,DAX,5 day,-5.259161,-0.896702,0.161798,1.224285,4.628697
3,2024-01-17,DAX,6 day,-6.282629,-1.007463,0.187498,1.435857,5.367396
4,2024-01-17,DAX,7 day,-6.735742,-1.054012,0.32619,1.557158,6.259934
0,2024-01-17,energy,36 hour,64.057678,70.027924,70.872932,75.010704,77.218788
1,2024-01-17,energy,40 hour,61.880836,66.906807,68.240898,72.040749,73.287773
2,2024-01-17,energy,44 hour,60.779305,62.762531,64.174416,68.024254,68.785774
3,2024-01-17,energy,60 hour,57.528931,60.668816,61.805847,64.52079,64.131546
4,2024-01-17,energy,64 hour,54.591389,57.760727,58.559002,60.839371,62.414742


In [15]:
df.to_csv('../../forecasts/20240117_JonSnow.csv',index=False)